# Практика построения регрессии

In [1]:
import numpy as np
import pandas as pd

from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

Давайте проанализируем данные опроса 4361 женщин из Ботсваны. О каждой из них мы знаем:

    -сколько детей она родила (признак ceb)
    -возраст (age)
    -длительность получения образования (educ)
    -религиозная принадлежность (religion)
    -идеальное, по её мнению, количество детей в семье (idlnchld)
    -была ли она когда-нибудь замужем (evermarr)
    -возраст первого замужества (agefm)
    -длительность получения образования мужем (heduc)
    -знает ли она о методах контрацепции (knowmeth)
    -использует ли она методы контрацепции (usemeth)
    -живёт ли она в городе (urban)
    -есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [2]:
rows = pd.read_csv('botswana.tsv',sep='\t', header=0)

In [3]:
rows.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [4]:
rows.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361 entries, 0 to 4360
Data columns (total 15 columns):
ceb         4361 non-null int64
age         4361 non-null int64
educ        4361 non-null int64
religion    4361 non-null object
idlnchld    4241 non-null float64
knowmeth    4354 non-null float64
usemeth     4290 non-null float64
evermarr    4361 non-null int64
agefm       2079 non-null float64
heduc       1956 non-null float64
urban       4361 non-null int64
electric    4358 non-null float64
radio       4359 non-null float64
tv          4359 non-null float64
bicycle     4358 non-null float64
dtypes: float64(9), int64(5), object(1)
memory usage: 511.2+ KB


In [5]:
rows.describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
count,4361.000000,4361.000000,4361.000000,4241.000000,4354.000000,4290.000000,4361.000000,2079.000000,1956.000000,4361.000000,4358.000000,4359.000000,4359.000000,4358.000000
mean,2.441642,27.405182,5.855996,4.615892,0.963252,0.577622,0.476726,20.686388,5.144683,0.516625,0.140202,0.701766,0.092911,0.275815
std,2.406861,8.685233,3.927075,2.219303,0.188164,0.493996,0.499515,5.002383,4.803028,0.499781,0.347236,0.457535,0.290341,0.446975
min,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,3.000000,3.000000,1.000000,0.000000,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,26.000000,7.000000,4.000000,1.000000,1.000000,0.000000,20.000000,6.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,4.000000,33.000000,8.000000,6.000000,1.000000,1.000000,1.000000,23.000000,8.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,13.000000,49.000000,20.000000,20.000000,1.000000,1.000000,1.000000,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
rows.religion.value_counts()
# Religion has 4 different values

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски? 

In [7]:
rows.dropna().shape[0]

1834

В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак x1 на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

создать новый бинарный признак x2={1,0,x1='не применимо',иначе; заменить "не применимо" в x1 на произвольную константу c, которая среди других значений x1 не встречается. Теперь, когда мы построим регрессию на оба признака и получим модель вида y=β0+β1x1+β2x2, на тех объектах, где x1 было измерено, регрессионное уравнение примет вид y=β0+β1x, а там, где x1 было "не применимо", получится y=β0+β1c+β2. Выбор c влияет только на значение и интерпретацию β2, но не β1.

Давайте используем этот метод для обработки пропусков в agefm и heduc.

Создайте признак nevermarr, равный единице там, где в agefm пропуски. Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице X будет мультиколлинеарность. Замените NaN в признаке agefm на cagefm=0. У объектов, где nevermarr = 1, замените NaN в признаке heduc на cheduc1=−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки). Сколько осталось пропущенных значений в признаке heduc?

In [8]:
rows.dropna().shape

(1834, 15)

In [9]:
rows['nevermarr'] = rows['agefm'].apply(lambda x: 1 if pd.isna(x) else 0)

In [10]:
rows.drop('evermarr', axis=1, inplace=True)

In [11]:
rows['agefm'] = rows.agefm.fillna(0)

In [12]:
rows.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [13]:
rows[pd.isna(rows['heduc'])]['nevermarr'].value_counts()

1    2282
0     123
Name: nevermarr, dtype: int64

Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения (cidlnchld=−1, cheduc2=−2 (значение -1 мы уже использовали), cusemeth=−1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [14]:
a = rows.heduc.to_numpy()
for n, i in enumerate(rows.heduc.to_numpy()):
    if n in rows[rows['nevermarr'] == 1]['heduc'].index:
        a[n] = -1

In [15]:
rows[rows['nevermarr'] == 1]['heduc'].index

Int64Index([   0,    3,   15,   16,   17,   24,   27,   28,   29,   31,
            ...
            4345, 4347, 4348, 4349, 4350, 4351, 4352, 4356, 4357, 4358],
           dtype='int64', length=2282)

In [16]:
values = 0

for i in range(rows.heduc.value_counts().shape[0]):
    values += rows.heduc.value_counts().iloc[i]
    
    
print(rows.shape[0] - values)

123


In [17]:
np.sum(pd.isna(rows['heduc']))

123

In [18]:
rows.columns

Index(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth',
       'agefm', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle',
       'nevermarr'],
      dtype='object')

In [19]:
print(np.sum(pd.isna(rows['heduc'])))
print(np.sum(pd.isna(rows['idlnchld'])))
print(np.sum(pd.isna(rows['usemeth'])))

123
120
71


In [20]:
rows['heduc'] = rows.heduc.fillna(-2)
rows['idlnchld'] = rows.idlnchld.fillna(-1)
rows['usemeth'] = rows.usemeth.fillna(-1)

In [21]:
data = rows.dropna()

In [22]:
data.shape

(4348, 15)

In [23]:
rows.shape

(4361, 15)

In [24]:
data.columns

Index(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth',
       'agefm', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle',
       'nevermarr'],
      dtype='object')

In [25]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [26]:
data['idlnchld_noans'] = 0
data.loc[data.idlnchld.isnull(), 'idlnchld_noans'] = 1

data['heduc_noans'] = 0
data.loc[data.heduc.isnull(), 'heduc_noans'] = 1

data['usemeth_noans'] = 0
data.loc[data.usemeth.isnull(), 'usemeth_noans'] = 1

data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R2? Округлите до трёх знаков после десятичной точки.

In [27]:
formula = 'ceb ~ ' + ' + '.join(data.columns[1:])
formula

'ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'

In [28]:
reg_m = smf.ols(formula, data=data)
fitted_m = reg_m.fit()
print(fitted_m.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.638
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     477.9
Date:                Wed, 22 Jul 2020   Prob (F-statistic):               0.00
Time:                        12:12:56   Log-Likelihood:                -7767.4
No. Observations:                4348   AIC:                         1.557e+04
Df Residuals:                    4331   BIC:                         1.568e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0

In [29]:
print(data.religion.value_counts())

spirit        1838
other         1076
protestant     989
catholic       445
Name: religion, dtype: int64


Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.

In [30]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted_m.resid, fitted_m.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [31]:
reg_m2 = smf.ols(formula, data=data)
fitted_m2 = reg_m2.fit(cov_type='HC1')
print(fitted_m2.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.638
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     380.7
Date:                Wed, 22 Jul 2020   Prob (F-statistic):               0.00
Time:                        12:12:56   Log-Likelihood:                -7767.4
No. Observations:                4348   AIC:                         1.557e+04
Df Residuals:                    4331   BIC:                         1.568e+04
Df Model:                          16                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0

Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [32]:
formula2 = 'ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle \
+ nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'
formula2

'ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'

In [33]:
reg_m3 = smf.ols(formula2, data=data)
fitted_m3 = reg_m3.fit()
print(fitted_m3.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.638
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     694.3
Date:                Wed, 22 Jul 2020   Prob (F-statistic):               0.00
Time:                        12:12:56   Log-Likelihood:                -7770.6
No. Observations:                4348   AIC:                         1.557e+04
Df Residuals:                    4336   BIC:                         1.564e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0350      0.197     -5.

In [34]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted_m3.resid, fitted_m3.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [35]:
reg_m4 = smf.ols(formula2, data=data)
fitted_m4 = reg_m4.fit(cov_type='HC1')
print(fitted_m4.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.638
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     547.4
Date:                Wed, 22 Jul 2020   Prob (F-statistic):               0.00
Time:                        12:12:57   Log-Likelihood:                -7770.6
No. Observations:                4348   AIC:                         1.557e+04
Df Residuals:                    4336   BIC:                         1.564e+04
Df Model:                          11                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0350      0.250     -4.

In [36]:
print('F=%f, p=%f, k1=%f' % reg_m2.fit().compare_f_test(reg_m4.fit()))

F=1.244235, p=0.285502, k1=5.000000


Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением cusemeth=−1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 5.5×10−8, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [37]:
formula3 = 'ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle \
+ nevermarr + idlnchld_noans + heduc_noans'
formula3

'ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans'

In [38]:
reg_m5 = smf.ols(formula3, data=data)
fitted_m5 = reg_m5.fit()
print(fitted_m5.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.625
Model:                            OLS   Adj. R-squared:                  0.624
Method:                 Least Squares   F-statistic:                     722.9
Date:                Wed, 22 Jul 2020   Prob (F-statistic):               0.00
Time:                        12:12:57   Log-Likelihood:                -7846.4
No. Observations:                4348   AIC:                         1.571e+04
Df Residuals:                    4337   BIC:                         1.578e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1593      0.200     -5.

In [39]:
print('F=%f, p=%.40f, k1=%f' % reg_m4.fit().compare_f_test(reg_m5.fit()))

F=153.919630, p=0.0000000000000000000000000000000000931962, k1=1.000000


Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.

In [40]:
print(fitted_m4.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.638
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     547.4
Date:                Wed, 22 Jul 2020   Prob (F-statistic):               0.00
Time:                        12:12:57   Log-Likelihood:                -7770.6
No. Observations:                4348   AIC:                         1.557e+04
Df Residuals:                    4336   BIC:                         1.564e+04
Df Model:                          11                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0350      0.250     -4.